In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.transforms import v2

In [ ]:
transform = v2.Compose(
    [v2.Resize((256, 256),antialias=True), # Scaling the images to 256x256 pixels
     v2.ToTensor(), # Converting the images to 5D tensors
     v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalizing the tensors to have a mean of 0.5 and standard deviation of 0.5

dataset = torchvision.datasets.ImageFolder(root='data', transform=transform)

batch_size = 20

np.random.seed(10)  # Assigned a seed for reproducibility of the model

# Spliting the dataset into two
# 80% of the dataset is the training set
# 2* % of the dataset is the testing set
num_samples = len(dataset)
num_train = int(0.8 * num_samples)
num_test = int(0.2 * num_samples)

trainset, testset = torch.utils.data.random_split(dataset, [num_train, num_test])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

# 6 classes for each species investigated
classes = ('Common Wheat','Aegilops cylindrica L.', 'Avena fatua L.', 'Bromus sterilis L.', 
           'Bromus tectorum L.', 'Echinochloa crus-galli (L.) Beauv')

print(f'Training and testing sets are separated')

In [ ]:
# Setting the class for specific CNN architecture
# Two convolutional layers each followed by a max pooling layer and three fully connected layers
class Net(nn.Module):
    
    # The constructor for the network 
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 62 * 62, 80)
        self.fc2 = nn.Linear(80, 42)
        self.fc3 = nn.Linear(42, 6)

    # Forward propagation
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x

net = Net() # Intitialization of the network

In [ ]:
#Loss function, backpropagation function, and  other layer-independent hyperparameters

lossFunc = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
epochs = 12

PATH = 'weed_classifier.pth' # Path to do the model

In [ ]:
# Looping over the dataset
for epoch in range(epochs): 

    current_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Getting the inputs
        inputs, labels = data

        # Zeroing the parameter gradients
        optimizer.zero_grad()

        # Forward and backward propagation, then optimizing
        outputs = net(inputs)
        loss = lossFunc(outputs, labels)
        loss.backward()
        optimizer.step()

        # Printing the loss statistics
        current_loss += loss.item()
        if i % 20 == 19: # printing every 20th batch
            print(f'[{epoch + 1}, {i + 1:3d}] loss: {current_loss / 2000:.4f}')
            current_loss = 0.0

print('Finished Training')

torch.save(net.state_dict(), PATH)